In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
OUTPUT_FOLDER = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/RGB_feature/labelled_images"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
# Loading stored reference features from CSV
def load_reference_features(csv_file):
    return pd.read_csv(csv_file, header=None).values.flatten()

In [ ]:
# Function to extract texture and color features
def extract_features(image_path, distance=5):
    img = cv2.imread(image_path)

    # Checking if the image is there
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return None
    img = cv2.resize(img, (128, 128))  # Resize for consistency

    # Use green channel (index 1 in BGR) instead of converting to grayscale
    green_channel = img[:, :, 1]  # BGR format in OpenCV, 1 is green

    # GLCM Texture Features using green channel
    glcm = graycomatrix(green_channel, distances=[distance], angles=[0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

    # Color Histogram Features (RGB)
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]).flatten()

    # Edge Detection (Canny) using green channel
    edges = cv2.Canny(green_channel, 100, 200)
    edge_density = np.sum(edges) / (128 * 128)  # Ratio of edge pixels

    return np.hstack([contrast, correlation, energy, homogeneity, hist, edge_density])

# Test with different distance values
def test_distances(image_path):
    distances = [1, 3, 5, 7, 10]  # Different distance values to test
    results = {}

    for dist in distances:
        features = extract_features(image_path, distance=dist)
        if features is not None:
            results[dist] = {
                'contrast': features[0],
                'correlation': features[1],
                'energy': features[2],
                'homogeneity': features[3],
                'edge_density': features[-1]
            }
            print(f"Distance {dist}:")
            print(f"Contrast: {features[0]:.4f}")
            print(f"Correlation: {features[1]:.4f}")
            print(f"Energy: {features[2]:.4f}")
            print(f"Homogeneity: {features[3]:.4f}")
            print(f"Edge Density: {features[-1]:.4f}")
            print("--------------------")

    return results

In [ ]:
def load_reference_features(feature_csv):
    try:
        df = pd.read_csv(feature_csv, header=None)
        print("Loaded CSV shape:", df.shape)
        if df.empty:
            raise ValueError("CSV file is empty!")
        features = df.values.flatten()
        print("Loaded features length:", len(features))
        if len(features) != 517:
            raise ValueError(f"Expected 517 features, got {len(features)}")
        return features
    except Exception as e:
        print(f"Error loading reference features: {e}")
        return None

def classify_and_rename_images(image_folder, feature_csv, output_folder, threshold=0.8, distance=5):
    reference_features = load_reference_features(feature_csv)
    if reference_features is None:
        print("Aborting classification due to reference feature loading failure!")
        return

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            image_path = os.path.join(image_folder, filename)
            features = extract_features(image_path, distance=distance)
            if features is None:
                print(f"Skipping {filename} - feature extraction failed")
                continue

            try:
                similarity = np.dot(reference_features, features) / (
                    np.linalg.norm(reference_features) * np.linalg.norm(features)
                )
                label = "1" if similarity >= threshold else "0"
                new_filename = f"{label}_{filename}"
                new_path = os.path.join(output_folder, new_filename)
                img = cv2.imread(image_path)
                if img is not None:
                    cv2.imwrite(new_path, img)
                    print(f"Processed: {filename} → {new_filename} (Similarity: {similarity:.2f})")
                else:
                    print(f"Failed to read image: {filename}")
            except ValueError as e:
                print(f"Error computing similarity for {filename}: {e}")

# Paths
image_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/splitted_images"
feature_file = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/RGB_feature/extracted_RGB_feature.csv"
output_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/RGB_feature/labelled_images"

# Classify images
classify_and_rename_images(image_folder, feature_file, output_folder, threshold=0.8, distance=5)

# Sync Drive
drive.flush_and_unmount()

Loaded CSV shape: (1, 517)
Loaded features length: 517
Processed: image_1_tile_0.png → 1_image_1_tile_0.png (Similarity: 0.95)
Processed: image_1_tile_1.png → 1_image_1_tile_1.png (Similarity: 0.95)
Processed: image_1_tile_2.png → 1_image_1_tile_2.png (Similarity: 0.87)
Processed: image_1_tile_3.png → 1_image_1_tile_3.png (Similarity: 0.83)
Processed: image_1_tile_4.png → 1_image_1_tile_4.png (Similarity: 0.91)
Processed: image_1_tile_5.png → 1_image_1_tile_5.png (Similarity: 0.88)
Processed: image_1_tile_6.png → 0_image_1_tile_6.png (Similarity: 0.70)
Processed: image_1_tile_7.png → 1_image_1_tile_7.png (Similarity: 0.80)
Processed: image_1_tile_8.png → 1_image_1_tile_8.png (Similarity: 0.93)
Processed: image_1_tile_9.png → 1_image_1_tile_9.png (Similarity: 0.96)
Processed: image_1_tile_10.png → 1_image_1_tile_10.png (Similarity: 0.95)
Processed: image_1_tile_11.png → 1_image_1_tile_11.png (Similarity: 0.86)
Processed: image_1_tile_12.png → 1_image_1_tile_12.png (Similarity: 0.84)
Pro

In [ ]:
!pip install scikit-image opencv-python pandas
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive # Make sure this is imported

In [ ]:
# Function to check accuracy against manually labeled images
def check_accuracy(manual_folder, predicted_folder):
    # Lists to store true and predicted labels
    y_true = []
    y_pred = []

    # Get manually labeled images
    manual_files = {f for f in os.listdir(manual_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))}

    # Get predicted labeled images
    predicted_files = {f for f in os.listdir(predicted_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))}

    # Compare labels
    for manual_file in manual_files:
        # Extract true label from filename (assuming format: "1_image.png" or "0_image.png")
        true_label = manual_file.split('_')[0]
        original_name = '_'.join(manual_file.split('_')[1:])  # Get original filename without label

        # Find corresponding predicted file
        predicted_name = f"1_{original_name}" if f"1_{original_name}" in predicted_files else f"0_{original_name}"
        if predicted_name in predicted_files:
            pred_label = predicted_name.split('_')[0]
            y_true.append(int(true_label))
            y_pred.append(int(pred_label))
        else:
            print(f"Warning: No prediction found for {manual_file}")

    # Compute accuracy
    if y_true and y_pred:
        accuracy = accuracy_score(y_true, y_pred)
        report = classification_report(y_true, y_pred, target_names=['Negative (0)', 'Positive (1)'])
        print(f"Accuracy: {accuracy:.4f}")
        print("Classification Report:\n", report)
    else:
        print("Error: No matching labels found for comparison!")

In [ ]:
output_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/RGB_feature/labelled_images"
manual_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Trial_Classification/manually_labelled_images"

print("Manual folder path:", manual_folder)

# Check if the directory exists
if not os.path.exists(manual_folder):
  print(f"Error: Manual folder not found: {manual_folder}")

#Check accuracy against manually labeled images
check_accuracy(manual_folder, output_folder)

Manual folder path: /content/gdrive/MyDrive/AI_WRM/Classification/Trial1_Classification/manually_labelled_images
Accuracy: 0.7768
Classification Report:
               precision    recall  f1-score   support

Negative (0)       0.61      0.92      0.73        37
Positive (1)       0.95      0.71      0.81        75

    accuracy                           0.78       112
   macro avg       0.78      0.81      0.77       112
weighted avg       0.83      0.78      0.78       112

